In [1]:
#!pip install mip
from mip import *
import numpy as np

In [2]:
def le_arquivo(entrada):
    with open(entrada,"r") as f:
        linha_1 = f.readline().split()
        num_var = int(linha_1[0]) #numero de variaveis
        num_restr = int(linha_1[1]) #numero de restricoes

        #print("Numero de variáveis: ", num_var)
        #print("Numero de restrições: ", num_restr)

        coef_FO = [ int(i) for i in f.readline().split()] #le os coefieicientes da primeira linha - FO
        #print("Coeficientes da função objetivo: ", coef_FO)

        coef_rest = []
        for i in range(num_restr):
            coef_rest.append([int(j) for j in f.readline().split()]) #le os coeficientes das restrições e armazena em uma lista

        #print("Coeficientes das restrições: ", coef_rest)
        
        return (num_var, num_restr, coef_FO, coef_rest) #armazena em uma tupla

In [3]:
def cria_modelo(num_var,num_restr,coef_FO,coef_restr):
    model = Model(sense=MAXIMIZE, solver_name=CBC)

    x = []
    for i in range(num_var): #cria as variáveis
        x.append(model.add_var(name=f"x_{i}", lb=0,ub=1,var_type=CONTINUOUS)) #Variáveis relaxadas 0 <= x <= 1


    FO = 0
    for i in range(num_var):
        FO += coef_FO[i] * x[i] #Escreve a funçao objetivo
    model.objective = FO #adiciona ao modelo

    
    #restrições LISTA DE RESTRIÇÕES
    for i in range(num_restr):
        LHS = 0
        for j in range(num_var):
            LHS += coef_restr[i][j] * x[j] #coef_rest da restrição i na variável j, vai até o penúltimo index 
        model += LHS <= coef_restr[i][num_var] #ultimo index é o RHS
        #print(LHS <= coef_restr[i][num_var])

    return model

In [4]:
def show_model(modelo):
    modelo.write("model.lp") # salva modelo em arquivo
    with open("model.lp", "r") as f: # lê e exibe conteúdo do arquivo
        print(f.read())

In [5]:
def bound(modelo, Zp):

  status = modelo.optimize() #Resolve o modelo

  for var in modelo.vars:
    print(f"{var.name}: {var.x}")
  print(f"Z*: {modelo.objective_value}")
  print("Zp = ", Zp)

  #Poda por inviabilidade:
  if status != OptimizationStatus.OPTIMAL:
    return "Poda por Inviabilidade"
  
  

  #Poda por limitante:
  if modelo.objective_value <= Zp:
    return "Poda por Limitante"

  
  cont_int = 0 #contador de variáveis inteiras
  for var in modelo.vars:
    if var.x == int(var.x):
      cont_int += 1

  #Poda por integralidade:
  if cont_int == dados_instancia[0]: #verifica se todas as variáveis são inteiras.
    return "Poda por Integralidade"
  

  return "Continua" #Quando as variáveis são fracionárias

In [6]:
def branch(modelo):
  
  #variavel escolhida vai ser a que estiver mais próxima de 0.5
  dist_vars = []
  for var in modelo.vars:
    dist_vars.append(abs(var.x - 0.5))
  #print(dist_vars)

  index = np.argmin(dist_vars) #retorna o index da variável mais próxima de 0.5
  #print("Index escolhido: ", index)
  branch_var = modelo.vars[index] #variável a ser ramificada
  print("Variavel a ser ramificada: ", branch_var)


  #ramificando o modelo
  modelo_left = modelo.copy()
  modelo_left += branch_var == 0 #adiciona restrição da variável = 0
  #show_model(modelo_left)

  modelo_right = modelo.copy()
  modelo_right += branch_var == 1 #adiciona restrição da variável = 1
  #show_model(modelo_right)
    

  return (modelo_left, modelo_right)


In [7]:
def branch_bound(modelo):
    
    Zp = -np.inf #definir o limite primal inicial
    best_solution = None

    #fila de modelos começa com o modelo original
    fila = [modelo] #problema com a relaxação linear

    while (len(fila)) > 0: #Enquanto houver modelos na fila
        print("Fila de modelos: ", fila)
        show_model(fila[0])
        

        bound_status = bound(fila[0], Zp) #verifica se o modelo é podado
        print(bound_status)

        if bound_status == "Poda por Inviabilidade": #Significa que o modelo eh inviavel e deve ser removido da fila
            fila.pop(0) #remove o modelo da fila


        if bound_status == "Poda por Integralidade":
            #So muda as variaveis forem todas inteiras e o valor da FO for maior que o limite primal
            if fila[0].objective_value > Zp:
                Zp = fila[0].objective_value
                best_solution = fila[0] #Melhor modelo até o momento
            fila.pop(0) #remove o modelo da fila

        elif bound_status == "Poda por Limitante":
            fila.pop(0)

        elif bound_status == "Continua":
            #Se o modelo nao for podado, ele eh ramificado
            modelo_left, modelo_right = branch(fila[0])
            
            fila.pop(0) #remove o modelo da fila
            fila.append(modelo_left) #adiciona o modelo ramificado na fila
            fila.append(modelo_right) #adiciona o modelo ramificado na fila
    return Zp,[x.x for x in best_solution.vars]

In [8]:
entrada = "instancia_1.txt"
dados_instancia = le_arquivo(entrada)

modelo = cria_modelo(*dados_instancia)
show_model(modelo)

branch_bound(modelo)

\Problem name: 

Minimize
OBJROW: -100 x_0 -80 x_1 -40 x_2 -30 x_3
Subject To
constr(0):  6 x_0 + 5 x_1 + 2 x_2 + 3 x_3 <= 10
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
End

Fila de modelos:  [<mip.model.Model object at 0x0000026229C2F610>]
\Problem name: 

Minimize
OBJROW: -100 x_0 -80 x_1 -40 x_2 -30 x_3
Subject To
constr(0):  6 x_0 + 5 x_1 + 2 x_2 + 3 x_3 <= 10
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
End

x_0: 1.0
x_1: 0.39999999999999997
x_2: 1.0
x_3: 0.0
Z*: 172.0
Zp =  -inf
Continua
Variavel a ser ramificada:  x_1
Fila de modelos:  [<mip.model.Model object at 0x0000026221F0B550>, <mip.model.Model object at 0x0000026229DC3490>]
\Problem name: 

Minimize
OBJROW: -100 x_0 -80 x_1 -40 x_2 -30 x_3
Subject To
constr(0):  6 x_0 + 5 x_1 + 2 x_2 + 3 x_3 <= 10
constr(1):  x_1 = 0
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
End

x_0: 1.0
x_1: 0.0
x_2: 1.0
x_3: 0.6666666666666665
Z*: 160.0
Zp =  -inf
Continua
Variavel 

(150.0, [0.0, 1.0, 1.0, 1.0])

In [9]:
entrada = "teste1.txt"
dados_instancia = le_arquivo(entrada)

modelo = cria_modelo(*dados_instancia)
show_model(modelo)

branch_bound(modelo)

\Problem name: 

Minimize
OBJROW: -2 x_0 -10 x_1 -8 x_2 -7 x_3 -10 x_4 -10 x_5 -6 x_6
Subject To
constr(0):  5 x_0 + 7 x_1 + 8 x_2 + x_3 + 7 x_4 + 5 x_5 + 6 x_6 <= 20
constr(1):  x_0 + 6 x_1 + 4 x_2 + 9 x_3 + 10 x_4 + 6 x_5 + 10 x_6 <= 30
constr(2):  4 x_0 + 4 x_1 + 4 x_2 + x_3 + 5 x_4 + 5 x_5 + 10 x_6 <= 40
constr(3):  3 x_0 + 10 x_1 + 8 x_2 + x_3 + 3 x_4 + 3 x_5 + 8 x_6 <= 30
constr(4):  10 x_0 + 8 x_1 + 9 x_2 + 9 x_3 + 7 x_4 + 6 x_5 + 10 x_6 <= 20
constr(5):  6 x_0 + 6 x_1 + 3 x_2 + 6 x_3 + 3 x_4 + 7 x_5 + 2 x_6 <= 80
constr(6):  7 x_0 + 10 x_1 + 7 x_2 + 8 x_3 + 7 x_4 + 8 x_5 + 7 x_6 <= 100
constr(7):  9 x_0 + 8 x_1 + x_2 + x_3 + 8 x_4 + 10 x_5 + 2 x_6 <= 90
constr(8):  x_0 + 5 x_1 + 3 x_2 + 10 x_3 + 2 x_4 + 4 x_5 + 9 x_6 <= 70
constr(9):  9 x_0 + 6 x_1 + x_2 + 4 x_3 + 7 x_4 + 5 x_5 + 10 x_6 <= 60
constr(10):  5 x_0 + 7 x_1 + 4 x_2 + 4 x_3 + 3 x_4 + 4 x_5 + 10 x_6 <= 40
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
En

(20.0, [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0])

In [10]:
entrada = "teste2.txt"
dados_instancia = le_arquivo(entrada)

modelo = cria_modelo(*dados_instancia)
show_model(modelo)

branch_bound(modelo)

\Problem name: 

Minimize
OBJROW: -7 x_0 -7 x_1 -7 x_2 -5 x_3 -8 x_4 -8 x_5 -9 x_6 -10 x_7 -7 x_8
Subject To
constr(0):  x_0 + 3 x_1 + x_2 + 3 x_3 + 3 x_4 + 7 x_5 + 2 x_6 + x_7 + 4 x_8 <= 80
constr(1):  7 x_0 + 6 x_1 + 10 x_2 + x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 7 x_7 + x_8 <= 90
constr(2):  3 x_0 + 2 x_1 + x_2 + 3 x_3 + 3 x_4 + 2 x_5 + x_6 + 6 x_7 + 5 x_8 <= 10
constr(3):  10 x_0 + 8 x_1 + 3 x_2 + 6 x_3 + 10 x_4 + 7 x_5 + 3 x_6 + 10 x_7 + 4 x_8 <= 30
constr(4):  2 x_0 + 8 x_1 + 6 x_2 + 5 x_3 + 6 x_4 + 6 x_5 + 9 x_6 + 7 x_7 + 2 x_8 <= 80
constr(5):  3 x_0 + 10 x_1 + x_2 + 9 x_3 + 2 x_4 + 7 x_5 + 7 x_6 + 9 x_7 + 10 x_8 <= 90
constr(6):  x_0 + 7 x_1 + 10 x_2 + 10 x_3 + 5 x_4 + 2 x_5 + 9 x_6 + 10 x_7 + 2 x_8 <= 20
constr(7):  10 x_0 + 3 x_1 + 2 x_2 + 3 x_3 + 10 x_4 + 2 x_5 + x_6 + 9 x_7 + 7 x_8 <= 10
constr(8):  2 x_0 + x_1 + 7 x_2 + 10 x_3 + x_4 + x_5 + 2 x_6 + x_7 + x_8 <= 50
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <=

(24.0, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0])

In [11]:
entrada = "teste3.txt"
dados_instancia = le_arquivo(entrada)

modelo = cria_modelo(*dados_instancia)
show_model(modelo)

branch_bound(modelo)

\Problem name: 

Minimize
OBJROW: -7 x_0 -9 x_1 -10 x_2 -3 x_3 -6 x_4 - x_5 -9 x_6 -8 x_7 -8 x_8
Subject To
constr(0):  2 x_0 + x_1 + 9 x_2 + 6 x_3 + 3 x_4 + 6 x_5 + 10 x_6 + 9 x_7 + x_8 <= 60
constr(1):  8 x_0 + 6 x_1 + 6 x_2 + 5 x_3 + 2 x_4 + 2 x_5 + 4 x_6 + 3 x_7 + 6 x_8 <= 80
constr(2):  8 x_0 + x_1 + 3 x_2 + 7 x_3 + x_4 + 4 x_5 + 8 x_6 + 3 x_7 + 4 x_8 <= 30
constr(3):  6 x_0 + 3 x_1 + 9 x_2 + 5 x_3 + 9 x_4 + 6 x_5 + 9 x_6 + 9 x_7 + 6 x_8 <= 40
constr(4):  10 x_0 + 8 x_1 + 8 x_2 + 7 x_3 + 10 x_4 + 10 x_5 + 9 x_6 + 9 x_7 + 3 x_8 <= 20
constr(5):  10 x_0 + 10 x_1 + 10 x_2 + 9 x_3 + 10 x_4 + x_5 + 8 x_6 + 3 x_7 + 10 x_8 <= 90
constr(6):  10 x_0 + 5 x_1 + 8 x_2 + 2 x_3 + 7 x_4 + 8 x_5 + 6 x_6 + 2 x_7 + 2 x_8 <= 90
constr(7):  7 x_0 + 9 x_1 + x_2 + 9 x_3 + 5 x_4 + 8 x_5 + 5 x_6 + 9 x_7 + 5 x_8 <= 80
constr(8):  2 x_0 + 4 x_1 + 6 x_2 + x_3 + 7 x_4 + 9 x_5 + 10 x_6 + x_7 + 7 x_8 <= 80
constr(9):  8 x_0 + 10 x_1 + 2 x_2 + 6 x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 4 x_7 + 9 x_8 <= 10
constr(10):  10 

(19.0, [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0])

In [12]:
entrada = "teste4.txt"
dados_instancia = le_arquivo(entrada)

modelo = cria_modelo(*dados_instancia)
show_model(modelo)

branch_bound(modelo)

\Problem name: 

Minimize
OBJROW: -9 x_0 -7 x_1 -10 x_2 -7 x_3 -9 x_4 -6 x_5 -8 x_6 -4 x_7 -9 x_8
Subject To
constr(0):  4 x_0 + 9 x_1 + 4 x_2 + x_3 + 9 x_4 + 6 x_5 + 3 x_6 + 6 x_7 + x_8 <= 40
constr(1):  3 x_0 + 7 x_1 + 8 x_2 + 7 x_3 + 6 x_4 + 3 x_5 + 5 x_6 + 9 x_7 + 4 x_8 <= 80
constr(2):  9 x_0 + 3 x_1 + 6 x_2 + 5 x_3 + 7 x_4 + x_5 + x_6 + 3 x_7 + 9 x_8 <= 40
constr(3):  5 x_0 + 9 x_1 + 6 x_2 + 5 x_3 + 9 x_4 + 7 x_5 + 8 x_6 + 7 x_7 + 8 x_8 <= 10
constr(4):  7 x_0 + 7 x_1 + 4 x_2 + x_3 + 3 x_4 + 4 x_5 + 8 x_6 + x_7 + 9 x_8 <= 10
constr(5):  x_0 + 6 x_1 + 6 x_2 + x_3 + 6 x_4 + 7 x_5 + 3 x_6 + 8 x_7 + 7 x_8 <= 10
constr(6):  6 x_0 + 6 x_1 + 8 x_2 + 6 x_3 + 10 x_4 + 8 x_5 + x_6 + 4 x_7 + 4 x_8 <= 70
constr(7):  9 x_0 + x_1 + 9 x_2 + 7 x_3 + 10 x_4 + 5 x_5 + 6 x_6 + 2 x_7 + 5 x_8 <= 10
constr(8):  2 x_0 + 7 x_1 + 6 x_2 + 5 x_3 + x_4 + x_5 + 9 x_6 + 2 x_7 + x_8 <= 20
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <

(10.0, [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])